Упражнение 1. Напишите программу Hello World, которая выводит номер процесса и Hello World. Запустите с 10 процессами. Объясните результат (почему не по порядку, при разных запусках порядок различается). Обратите внимание на использование MPI.

In [ ]:
!pip install mpi4py

     |████████████████████████████████| 2.5 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185307 sha256=56d6bf5cbd8e9aba8240861343ac91c933ccd2d9f6d794a70ac7a5af5d27b77f
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


In [ ]:
!mpiexec -n 5 --allow-run-as-root python -m mpi4py /content/task1.py

Hello world  3
Hello world  4
Hello world  2
Hello world  1
Hello world  0


Упражнение 2.Напишите программу MPI, которая определяет две переменные с плавающей точкой a и b. Используйте ранг:

-- выведите a-b, если ранг равен 0

-- выведите a+b, если ранг равен 1

-- выведите a*b, если ранг равен 2

Запустите программу и убедитесь, что она работает корректно. 

Упражнение 3. Запустите код ниже, убидитесь в его работоспособности.

In [ ]:
from mpi4py import MPI
import sys

n_ranks = MPI.COMM_WORLD.Get_size()
if n_ranks != 2:
    print("This example requires exactly two ranks", n_ranks)
    sys.exit(1)

rank = MPI.COMM_WORLD.Get_rank()

if rank == 0:
    message = "Hello, world!"
    MPI.COMM_WORLD.send(message, dest=1, tag=0)

if rank == 1:
    message = MPI.COMM_WORLD.recv(source=0, tag=0)
    print(message)

This example requires exactly two ranks 1


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
mpiexec -n 3 python program_name.py

Пример запуска программы на трех процессах

Измените приведенный выше пример, чтобы он работал с любым количеством рангов. Соедините четные ранги с нечетными и пусть каждый четный ранг отправляет сообщение соответствующему нечетному рангу.

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

a = 3.14
b = 2.71

if rank % 2 == 0:
    target_rank = rank + 1
    comm.send(a - b, dest=target_rank)
elif rank % 2 == 1:
    source_rank = rank - 1
    result = comm.recv(source=source_rank)
    print(f"Rank {rank}: a - b = {result}")

comm.Barrier() 
if rank % 2 == 0:
    target_rank = rank + 1
    comm.send(a + b, dest=target_rank)
elif rank % 2 == 1:
    source_rank = rank - 1
    result = comm.recv(source=source_rank)
    print(f"Rank {rank}: a + b = {result}")

comm.Barrier() 
if rank % 2 == 0:
    target_rank = rank + 1
    comm.send(a * b, dest=target_rank)
elif rank % 2 == 1:
    source_rank = rank - 1
    result = comm.recv(source=source_rank)
    print(f"Rank {rank}: a * b = {result}")


В программе мы получаем количество процессов, каждый четный процесс отправляет сообщение соответствующему нечетному процессу с результатом вычисления a - b, затем мы синхронизируем процессы, каждый четный процесс отправляет сообщение соответствующему нечетному процессу с результатом вычисления a + b, затем снова синхронизируем процессы, и каждый четный процесс отправляет сообщение соответствующему нечетному процессу с результатом вычисления a * b. Нечетные процессы принимают эти сообщения и выводят результаты на экран.

Упражнение 4. Запустите этот код и посмотрите, что произойдет.
Как бы вы изменили код, чтобы решить проблему?

In [ ]:
from mpi4py import MPI
import sys

n_numbers = 10000

rank = MPI.COMM_WORLD.Get_rank()
n_ranks = MPI.COMM_WORLD.Get_size()

if n_ranks != 2:
    print("This example requires exactly two ranks")
    sys.exit(1)

if rank == 0:
    neighbour = 1
else:
    neighbour = 0

send_message = []
for i in range(n_numbers):
    send_message.append(i)

MPI.COMM_WORLD.send(send_message, dest=neighbour, tag=0)

recv_message = MPI.COMM_WORLD.recv(source=neighbour, tag=0)
print("Message received by rank", rank)

ModuleNotFoundError: ignored

In [ ]:
!mpiexec -n 2 --allow-run-as-root python -m mpi4py /content/task4.py

Проблема заключается в том, что при выполнении данного кода возможна ситуация, когда процесс с рангом 0 будет ждать сообщение от процесса с рангом 1, тогда как процесс с рангом 1 уже завершил свою работу и не отправил сообщение. Чтобы исправить проблему можно использовать метод MPI.COMM_WORLD.Iprobe(), который проверяет, есть ли доступные сообщения в буфере для приема. Если сообщения нет, то метод возвращает значение False, а если сообщение есть, то метод возвращает значение True и объект, который можно принять с помощью метода MPI.COMM_WORLD.recv().

In [ ]:
from mpi4py import MPI
import sys

n_numbers = 10000

rank = MPI.COMM_WORLD.Get_rank()
n_ranks = MPI.COMM_WORLD.Get_size()

if n_ranks != 2:
    print("This example requires exactly two ranks")
    sys.exit(1)

if rank == 0:
    neighbour = 1
else:
    neighbour = 0

send_message = []
for i in range(n_numbers):
    send_message.append(i)

MPI.COMM_WORLD.send(send_message, dest=neighbour, tag=0)

if rank == 0:
    while True:
        flag = MPI.COMM_WORLD.Iprobe(source=neighbour, tag=0)
        if flag:
            recv_message = MPI.COMM_WORLD.recv(source=neighbour, tag=0)
            break

    print("Message received by rank", rank)
else:
    print("Message sent by rank", rank)


Упражнение 5. В упр. 1 мы написали программу hello world, в которой каждый ранг отправляет сообщение рангу 0. Перепишите, используя gather.

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


msg = "Hello from process {}".format(rank)

if rank == 0:
    recv_msg = [""] * size
else:
    recv_msg = None
    
comm.Gather(msg, recv_msg, root=0)

if rank == 0:
    for i in range(size):
        print(recv_msg[i])


В этом примере каждый процесс отправляет свою строку "Hello from process {rank}" на процесс с рангом 0, используя функцию MPI_Gather. Процесс с рангом 0 собирает все сообщения и выводит их в порядке их рангов.

Упражнение 6. Следующая программа создает массив с именем vector, который содержит список n_numbers для каждого ранга. Первый ранг содержит числа от 1 до n_numbers, второй ранг — от n_numbers до 2*n_numbers2 и так далее. Затем он вызывает функции find_max и find_sum, которые должны вычислить сумму и максимум вектора.
Эти функции не реализуются параллельно и возвращают только сумму и максимум локальных векторов. Измените функции find_sum и find_max, чтобы они работали правильно параллельно, используя MPI_Reduce или MPI_Allreduce.

In [ ]:
from mpi4py import MPI

def find_sum(vector):
    my_sum = 0.0
    for i in range(len(vector)):
        my_sum += vector[i]
    return my_sum

def find_maximum(vector):
    my_max = 0.0
    for i in range(len(vector)):
        if vector[i] > my_max:
            my_max = vector[i]
    return my_max

n_numbers = 1024

rank = MPI.COMM_WORLD.Get_rank()

my_first_number = n_numbers*rank

vector = []
for i in range(n_numbers):
    vector.append(float(my_first_number + i))

my_sum = find_sum(vector)
print("The sum of the numbers is", my_sum)

my_max = find_maximum(vector)
print("The largest number is", my_max)

ModuleNotFoundError: ignored

In [ ]:
from mpi4py import MPI

def find_sum(vector):
    my_sum = 0.0
    for i in range(len(vector)):
        my_sum += vector[i]
    total_sum = MPI.COMM_WORLD.allreduce(my_sum, op=MPI.SUM)
    return total_sum

def find_maximum(vector):
    my_max = 0.0
    for i in range(len(vector)):
        if vector[i] > my_max:
            my_max = vector[i]
    global_max = MPI.COMM_WORLD.allreduce(my_max, op=MPI.MAX)
    return global_max

n_numbers = 1024

rank = MPI.COMM_WORLD.Get_rank()

my_first_number = n_numbers*rank

vector = []
for i in range(n_numbers):
    vector.append(float(my_first_number + i))

my_sum = find_sum(vector)
print("The sum of the numbers is", my_sum)

my_max = find_maximum(vector)
print("The largest number is", my_max)